# GoogLeNet - Inception 모듈 구현

## Naive 버전

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, concatenate

model_input = Input(shape=(28,28,3))
pre = Dense(192)(model_input)
conv1 = Conv2D(64, 1, padding='same')(pre)
conv2 = Conv2D(128, 3, padding='same')(pre)
conv3 = Conv2D(32, 5, padding='same')(pre)
pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre)
model_output = concatenate([ conv1, conv2, conv3, pool])

model = Model(inputs=model_input, outputs=model_output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 28, 28, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 28, 28, 192)       │             768 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 28, 28, 64)        │          12,352 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 28, 28, 128)       │         221,312 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 28, 28, 32)        │         153,632 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 28, 28, 192)       │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 28, 28, 416)       │               0 │ conv2d[0][0],              │
│                               │                           │                 │ conv2d_1[0][0],            │
│                               │                           │                 │ conv2d_2[0][0],            │
│                               │                           │                 │ max_pooling2d[0][0]        │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 388,064 (1.48 MB)

 Trainable params: 388,064 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

## 1x1 Conv층을 추가한 Bottle Neck 구조

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras.layers import MaxPooling2D, concatenate

model_input = Input(shape=(28,28,3))
pre = Dense(192)(model_input)

conv1 = Conv2D(64, 1, padding='same')(pre)

conv1_2 = Conv2D(96, 1, padding='same')(pre)
conv2 = Conv2D(128, 3, padding='same')(conv1_2)

conv1_3 = Conv2D(16, 1, padding='same')(pre)
conv3 = Conv2D(32, 5, padding='same')(conv1_3)

pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre)
conv1_4 = Conv2D(32, 1, padding='same')(pool)
model_output = concatenate([ conv1, conv2, conv3, conv1_4])

model = Model(inputs=model_input, outputs=model_output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 28, 28, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 28, 28, 192)       │             768 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 28, 28, 96)        │          18,528 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_6 (Conv2D)             │ (None, 28, 28, 16)        │           3,088 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 28, 28, 192)       │               0 │ dense_1[0][0]              │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 28, 28, 64)        │          12,352 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 28, 28, 128)       │         110,720 │ conv2d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_7 (Conv2D)             │ (None, 28, 28, 32)        │          12,832 │ conv2d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_8 (Conv2D)             │ (None, 28, 28, 32)        │           6,176 │ max_pooling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 28, 28, 256)       │               0 │ conv2d_3[0][0],            │
│                               │                           │                 │ conv2d_5[0][0],            │
│                               │                           │                 │ conv2d_7[0][0],            │
│                               │                           │                 │ conv2d_8[0][0]             │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 164,464 (642.44 KB)

 Trainable params: 164,464 (642.44 KB)

 Non-trainable params: 0 (0.00 B)